In [122]:
import pandas as pd

In [123]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
week_nums = [190601, 190608, 190615, 190622, 190629, 190706, 190713, 190720, 190727, 190803, 190810, 190817, 190824, 190831, 190907, 190914, 190921, 190928]
df = get_data(week_nums)

In [124]:
#clean column names
df.rename(columns={'C/A': 'c_a', 'UNIT': 'unit', 'SCP': 'scp', 'STATION': 'station', 'LINENAME': 'linename', 'DIVISION': 'division', 'DATE': 'date','TIME': 'time', 'DESC': 'desc', 'ENTRIES': 'entries', df.columns[10]: 'exits'}, inplace=True)

#keep only subway divisions
df = df[(df['division'] == 'BMT') | (df['division'] == 'IND') | (df['division'] == 'IRT')]

#Add formatted timestamp column
df['timestamp'] = pd.to_datetime(df['date'] + " " + df['time'])

#add unique turnstile id
df['unique_turnstile_id'] = df['c_a'] + df['unit'] + ' ' + df['station'] + ' ' + df['scp']

#add day of week column
df['day_of_week'] = df.timestamp.dt.day_name()

df = df.reset_index()

In [125]:
#df.desc.value_counts()

In [126]:
# Get rid of duplicate entries caused by "RECOVER AUD" in desc column, if any
df.sort_values(["c_a", "unit", "scp", "station", "timestamp"], 
                          inplace=True, ascending=False)
df.drop_duplicates(subset=["c_a", "unit", "scp", "station", "timestamp"], inplace=True)

In [127]:
#keep only needed columns
df = df.loc[:,['station', 'entries', 'exits', 'timestamp', 'unique_turnstile_id']]

In [128]:
#calculate actual volumes for each record (record minus previous record)
df['diff_entries'] = abs(df.entries.diff())
df['diff_exits'] = abs(df.exits.diff())
df['total_traffic'] = df.diff_entries + df.diff_exits

#record index in new column (needed to identify first instance of turnstile data)
df['record_index'] = df.index
#df

In [129]:
#identify first rows of unique turnstile data
df_first_rows = df.groupby('unique_turnstile_id').first()

#index of records of first turnstile rows
list_index = df_first_rows.record_index.values

#drop first rows from main df
df.drop(list_index, inplace=True)

In [130]:
#keep only data with traffic > 0 amd < 14400 (1 person per second)
df = df[(df['total_traffic'] > 0) & (df['total_traffic'] < 14400)]

In [131]:
df = df.reset_index()


In [132]:
#drop record index column. no longer applies after re-indexing
df.drop('record_index', axis=1)

,index,station,entries,exits,timestamp,unique_turnstile_id,diff_entries,diff_exits,total_traffic
0,3441723,EASTCHSTER/DYRE,1559926981,1728192971,2019-09-27 17:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,51.0,81.0,132.0
1,3441722,EASTCHSTER/DYRE,1559927099,1728192888,2019-09-27 13:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,118.0,83.0,201.0
2,3441721,EASTCHSTER/DYRE,1559927163,1728192863,2019-09-27 09:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,64.0,25.0,89.0
3,3441720,EASTCHSTER/DYRE,1559927497,1728192834,2019-09-27 05:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,334.0,29.0,363.0
4,3441719,EASTCHSTER/DYRE,1559927501,1728192834,2019-09-27 01:00:00,R730R431 EASTCHSTER/DYRE 00-00-04,4.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...
3006302,4,59 ST,7072182,2396555,2019-05-25 16:00:00,A002R051 59 ST 02-00-00,274.0,38.0,312.0
3006303,3,59 ST,7071943,2396500,2019-05-25 12:00:00,A002R051 59 ST 02-00-00,239.0,55.0,294.0
3006304,2,59 ST,7071864,2396443,2019-05-25 08:00:00,A002R051 59 ST 02-00-00,79.0,57.0,136.0
3006305,1,59 ST,7071844,2396415,2019-05-25 04:00:00,A002R051 59 ST 02-00-00,20.0,28.0,48.0


In [133]:
df_by_station = df.groupby(["unique_turnstile_id", "timestamp"]).agg({'total_traffic': 'sum'}).reset_index().sort_values(by="total_traffic", ascending=False)
df_by_station

,unique_turnstile_id,timestamp,total_traffic
1309875,N324R018 JKSN HT-ROOSVLT 00-00-01,2019-08-21 08:00:00,13979.0
1318309,N324R018 JKSN HT-ROOSVLT 00-06-02,2019-08-21 04:00:00,12872.0
1304576,N323R018 JKSN HT-ROOSVLT 01-00-00,2019-08-21 16:00:00,11881.0
1315283,N324R018 JKSN HT-ROOSVLT 00-03-03,2019-08-21 00:00:00,11819.0
2342899,R243R049 51 ST 00-03-03,2019-07-27 16:00:00,11619.0
...,...,...,...
2257580,R227R131 23 ST 00-05-00,2019-06-12 05:00:00,1.0
2257582,R227R131 23 ST 00-05-00,2019-06-13 17:00:00,1.0
861130,N067R012 34 ST-PENN STA 00-03-05,2019-08-26 11:30:07,1.0
2257584,R227R131 23 ST 00-05-00,2019-06-15 13:00:00,1.0
